In [7]:
# Basic Libraries

import pandas as pd
import numpy as np
import scipy.signal

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV


In [8]:
# Libraries for Classification and building Models
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical 
from keras.preprocessing.image import ImageDataGenerator
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

In [9]:
# Project Specific Libraries

import os
import librosa
import librosa.display
import glob 

###  MFCC

In [10]:
# Extracting MFCC's For every audio file
import pandas as pd
import os
import librosa

audio_dataset_path='audio/'
metadata=pd.read_csv('metadata/ff1010bird_metadata_2018.csv')
metadata.head()

,itemid,datasetid,fold,hasbird
0,55.wav,ff1010bird,1,0
1,87.wav,ff1010bird,1,0
2,99.wav,ff1010bird,1,0
3,100.wav,ff1010bird,1,1
4,104.wav,ff1010bird,1,0


In [11]:
N_MELS = 40
FFT_POINTS = 882 * 2
SR = 44100
HAMMING_SIGNAL = scipy.signal.hamming
FRAMES = 500


In [12]:
# creating a function that extracts and lable the file 
# normalising the audio 

def feature_extract(audio_dataset_path, data_fold, data_id, data_labels):
    feature_file = []
    label_file = []

    for i in range(len(data_id)):
        #file_name = os.path.join(os.path.abspath(audio_dataset_path), 'fold'+str(row["fold"])+'/',str(row["itemid"]))
        file_name = audio_dataset_path + 'fold' + str(data_fold[i]) + '/' + str(data_id[i])
        [wave, _] = librosa.core.load(file_name ,sr=SR)  # read wav file (fs = 44.1 kHz)
        wave = librosa.stft(wave, FFT_POINTS, win_length=FFT_POINTS,
                            window=HAMMING_SIGNAL)  # STFT computation (fft_points = 882*2, overlap= 50%, analysis_window=40ms)
        wave = np.abs(wave) ** 2
        spectrogram = librosa.feature.melspectrogram(S=wave, n_mels=N_MELS)  # mel bands (40)
        spectrogram = librosa.power_to_db(spectrogram, ref=np.max)
        norm_spectrogram = spectrogram - np.amin(spectrogram)
        norm_spectrogram = norm_spectrogram / float(np.amax(norm_spectrogram))

        if int(norm_spectrogram.shape[1]) < FRAMES:  # 10 sec samples gives 500 frames
            z_pad = np.zeros((N_MELS, FRAMES))
            z_pad[:, :-(FRAMES - norm_spectrogram.shape[1])] = norm_spectrogram
            feature_file.append(z_pad)
        else:
            img = norm_spectrogram[:, np.r_[0:FRAMES]]  # final_shape = 40*500
            feature_file.append(img)

        if len(data_labels) > 0:
            label_file.append(data_labels[i])

    feature_file = np.array(feature_file)
    feature_file = np.reshape(feature_file, (len(data_id), N_MELS, FRAMES, 1))

    if len(data_labels) > 0:  # In case of training data
        label_file = np.array(label_file)
        return feature_file, label_file
    else:  # In case of testing data
        return feature_file

In [13]:
#extract the feautures of x and y and save it into a file
X,y = feature_extract(audio_dataset_path, metadata['fold'], metadata['itemid'], metadata['hasbird'])
np.save('features_data_X', X)
np.save('features_data_y', y)

In [14]:
#load the extracted feautures
X= np.load('features_data_X.npy')
y= np.load('features_data_y.npy')

In [15]:
X.shape

(15690, 40, 500, 1)

In [16]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [17]:
y.shape

(15690, 2)

### Splitting dataset 

In [21]:
#splitting the dataset to training and validation
X_trainset, X_val, y_trainset, y_val = train_test_split(X, y, random_state = 1)

In [28]:
X_trainset.shape

(11767, 40, 500, 1)

In [23]:
X_val.shape

(3923, 40, 500, 1)

In [24]:
# splitting the training dataset to obtain some test set for the testing phase
X_train, X_test, y_train, y_test = train_test_split(X_trainset, y_trainset, random_state = 1)

In [25]:
X_train.shape

(8825, 40, 500, 1)

In [26]:
X_test.shape

(2942, 40, 500, 1)

In [29]:
X_val.shape

(3923, 40, 500, 1)

In [22]:
input_dim = (40, 500, 1)

### CNN Model

In [18]:
model = Sequential()
model.add(Conv2D(48, kernel_size=3, activation='relu', input_shape=input_dim))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(48, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(48, kernel_size=5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(96, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(96, kernel_size=3, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Conv2D(96, kernel_size=5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Flatten())

model.add(Dense(384, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(2, activation='sigmoid'))

In [19]:
optimizer = keras.optimizers.Adam(lr=3e-4)
model.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [20]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 38, 498, 48)       480       
_________________________________________________________________
batch_normalization (BatchNo (None, 38, 498, 48)       192       
_________________________________________________________________
dropout (Dropout)            (None, 38, 498, 48)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 36, 496, 48)       20784     
_________________________________________________________________
batch_normalization_1 (Batch (None, 36, 496, 48)       192       
_________________________________________________________________
dropout_1 (Dropout)          (None, 36, 496, 48)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 18, 248, 48)       5

In [21]:
def data_augmentation():
    gen = ImageDataGenerator(
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=True)
    return gen

In [22]:
gen = data_augmentation()

In [ ]:
#adding the learning rate
def scheduler(epoch, lr):
  if epoch < 10:
    return lr
  else:
    return lr * tf.math.exp(-0.1)

In [23]:
callback = [keras.callbacks.ReduceLROnPlateau(scheduler, verbose=1),
            keras.callbacks.ModelCheckpoint(filepath='saved_models/audio_classification.hdf5', monitor='val_loss', verbose=1,
                                            save_best_only=True),
            keras.callbacks.LearningRateScheduler(lambda x: 1e-3 * 0.7 ** x),
            keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=3)]

In [24]:
EPOCHS = 50

In [25]:
hist = model.fit(gen.flow(X_train, y_train, batch_size=64),
                              epochs=EPOCHS,
                              steps_per_epoch=X_train.shape[0] // 64,
                              validation_data=(X_test, y_test),
                              callbacks=callback,
                              verbose=1)

Epoch 1/50
 2/90 [..............................] - ETA: 1:03 - loss: 2.1001 - accuracy: 0.5493

KeyboardInterrupt: 

In [ ]:
#plot graph

plt.plot(hist.history['loss'], label='train')
plt.plot(hist.history['val_loss'], label='val')
plt.title('Loss and Validation Loss')
plt.xlabel('epoch')
plt.ylabel('Accuracy')
plt.legend();
plt.show()

plt.plot(hist.history['accuracy'], label='train')
plt.plot(hist.history['val_accuracy'], label='val')
plt.title(' Accuracy and Validation Accuracy ')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend();
plt.show()